<a href="https://colab.research.google.com/github/sharmapratik88/LearningAI/blob/main/DeepLearning.AI/02_Building_Systems_with_OpenAI_API/01_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building Systems with the ChatGPT API
A [DeepLearning.AI](https://www.deeplearning.ai/short-courses/) short course taught by Andrew NG and Isa Fulford (OpenAI).

Coverage:
- Building a system that requires more than a single prompt or a single call to an LLM or a large language model
- How do you evaluate the input to ensure it doesn't contain problematic content, such as hateful speech, or isn't providing inaccurate or inappropriate answers?
- How do we sequentially process the user input in multiple steps to get to the final output shown to the user?
- Split complex tasks into a pipeline of subtasks using multistage prompts.

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Setting the current working directory
import os; os.chdir('/content/drive/MyDrive/Learnings/DeepLearning.AI/Building_Systems_with_ChatGPT_API')

In [3]:
# Import packages/libraries
!pip install -q openai

from helperFunc import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


## Large Language Model (LLMs)

LLMs are sophisticated computer programs capable of generating and analyzing human-quality text and have revolutionized various aspects of Natural Language Processing (NLP).

Read more at [ChatGPT Prompt Engineering & LLMs](https://pratikdsharma.com/chatgpt-prompt-engineering/).

### Answering how do they work?
- For e.g., text generation, prompt: "I love eating," and LLM generates the rest, which might be "bagels with cream cheese or out with friends."

- The main tool to train an LLM is supervised learning (x -> y), which predicts the next word repeatedly. For example, your training sets contain a lot of text data, which says, "My favorite food is a bagel with cream cheese and lox." Then, when asking LLM the above prompt again, it might respond to the next word as "bagel".

- Two major LLMs:
  - base LLMs: predicts next word, based on text training data
  - instruction LLMs: Try to follow instructions
  - Read more at [ChatGPT Prompt Engineering & LLMs](https://pratikdsharma.com/chatgpt-prompt-engineering/).

  **How do we go from base to instruction-tuned LLMs?**
  - Train a base LLM on loads of data
  - Further, train the model:
	- Fine-tune on examples of where the output follows input instructions.
   - Obtain human ratings of the quality of different LLM outputs based on whether it is helpful, honest, or harmless.
	- Tune LLM to increase the probability that it generates more highly rated outputs (using RHLF: Reinforcement Learning from Human Feedback).
	- Base LLM training can take months. For e.g., the process of going from the Base LLM to the Instruction Tuned LLM can be done in maybe days on a much more modest size dataset and much more modest size computational resources.

In [4]:
print(get_completion('What is the capital of France?'))

The capital of France is Paris.


In [5]:
print(get_completion('Take the letters in lollipop and reverse them!'))

The reversed letters of "lollipop" are "pillipol".


#### Importance of Tokens
  - LLM will take sequences of characters, like "Learning new things is fun!" and group the characters together to form tokens that comprise commonly occurring sequences of characters. Since each is a fairly common word, each token in the earlier example will correspond to one word or word in a space.
  - In example such as "Prompting as powerful developer tool", it will be actually broken down to three tokens with "prom", "pt", and "ing" because those three are commonly occuring sequences of letters.
  - For word lollipop, the tokenizer will break this down into three tokens, "l", "oll", and "ipop".

In [6]:
print(get_completion('Take the letters in l-o-l-l-i-p-o-p and reverse them!'))

p-o-p-i-l-l-o-l


In the above example, each letter was accompanied by dashes in between the letters, LLM tokenizes each of these characters into an individual toekn, making it easier for it to see the individual letters and print them out in reverse order.

For English language input, 1 token is around 4 characters or 3/4 of a word.

**Token limits:**
- Different models have different limits on the number of token in input (aka context) + output (aka completion)
- For e.g. for gpt3.5-turbo is ~4000 tokens (input + output)
- For an input context that is much longer than this, it will either throw an exception or generate an error.
- The limit for a given API request is the combination of the prompt and the completion. For example, if the prompt is 3,000 tokens, the completion cannot be more than around 1,000 tokens.


**Message roles:**
- System: sets tone/behavior of assistant
- user: prompts
- assistant: LLM response

In [7]:
messages =  [
    {'role':'system',
     'content':"""You are an assistant who
                  responds in the style of Dr Seuss.
                """
                  },
    {'role':'user',
     'content':"""write me a very short poem
                    about a happy carrot.
                """
                    },
]

res = get_completion_from_messages(messages, temp=1)
print(f'Response: {res}')

Response: In a garden so bright and cheery,
Lived a carrot, oh so merry!
With a vibrant orange hue,
And a smile that just grew and grew.

With each sunny day that passed,
This happy carrot danced and laughed.
Roots deeply planted in the ground,
Its joyous nature could easily be found.

From leafy greens to tender core,
This carrot radiated joy and more.
A crunchy delight in every bite,
Bringing happiness day and night.

So remember, dear friend, from this brief tale,
To be like the carrot, never fail.
Choose to bring happiness and cheer,
Spread joy to all, far and near!


In [8]:
# setting response length as a system message!
messages =  [
    {'role':'system',
     'content':"""You are an assistant who
                  responds in the style of Dr Seuss.
                  Your response must be one sentence long.
                """
                  },
    {'role':'user',
     'content':"""write me a very short poem
                    about a happy carrot
                """
                    },
]

res = get_completion_from_messages(messages, temp=1)
print(f'Response: {res}')

Response: In the garden, a carrot so orange and bright, filled with joy and delight.


In [9]:
# get the token counts!
messages =  [
    {'role':'system',
     'content':"""You are an assistant who
                  responds in the style of Dr Seuss.
                  Your response must be one sentence long.
                """
                  },
    {'role':'user',
     'content':"""write me a very short poem
                    about a happy carrot
                """
                    },
]

res, token_dict = get_completion_with_token_count(messages, temp=1)
print(f'Response: {res}')
print(f'\n\nToken Count Dictionary: {token_dict}')

Response: A happy carrot so orange and plump, makes tummies dance and hearts jump!


Token Count Dictionary: {'prompt_tokens': 51, 'completion_tokens': 18, 'total_tokens': 69}


### How Prompting is Revolutionizing AI Application Development!

In a traditional supervised ML workflow, a sentiment classifier on restaurant review might take maybe
- 1 month to get labeled data!
- 3 months to train model on data, choosing appropriate model, tune the model and then evaluating it!
- 3 months to find a cloud service, upload model on cloud and then run & call model!

whereas with prompting-based ML
- minutes/hours (if it requires iterative prompting) to get effective prompt!
- hours to get run using API calls and start making calls to the model
- minutes/hours to call the model and start making inferences



